In [77]:
import re #18/01/2023 works!
import pandas as pd
import numpy as np
import pdfplumber
import shutil
import os, glob

dir_remov = ["C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/", 
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/Transporeon_split_1",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/", 
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/",
             "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/"]
for dir_remo in dir_remov:
    for file_remov in glob.glob(dir_remo +"*.pdf"):
        os.remove(file_remov)

list_of_files = glob.glob('C:/Users/john.tan/Downloads/*.pdf') # *.csv
latest_file = max(list_of_files, key=os.path.getctime) #get latest files
shutil.move(latest_file, r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/")

from PyPDF2 import PdfWriter, PdfReader #split pdf files
from PyPDF2 import PdfWriter, PdfReader
path_file = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transports_Number_6.pdf"
file_out = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/Transport"
inputpdf = PdfReader(open(path_file, "rb"))

for i in range(len(inputpdf.pages)):
    output = PdfWriter()
    output.add_page(inputpdf.pages[i])
    with open(file_out + "_%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)
        
substr = 'Transporeon-ID' #remove pdf w/o 'Transporeon-ID'
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split/"
for file in glob.glob(path_dir+"*.pdf"):
    file = str(file)
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    
    if text.find(substr) !=-1:
        path_from = file
        #print(file)
        path_dest = 'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split_1/'
        shutil.move(path_from, path_dest)
        pdf.close()

        
regexp = re.compile("Origin(.*)$", re.MULTILINE)
path_dir_1 = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Transporeon_split_1/"
path_in = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #move pdf to 'Chawan', 'Vopak'
for file in glob.glob(path_dir_1 +"*.pdf"):
    file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori in ['CHAWAN', 'VOPAK']:
            shutil.move(path_from, path_in)
            pdf.close()
    except:
        pass

In [78]:
dir = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #regex rename pdf
regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
regexp_2 = re.compile(r'(?s)Shipping number (BOL) (.*?)\n[^\S\r\n]*Gross ', re.MULTILINE)
regexp_3 = re.compile(r'Shipping number (BOL) (\d+)', re.MULTILINE)

for file in os.listdir(dir):
    f = os.path.join(dir, file) # iterate over files in dir
    if os.path.isfile(f): # checking if it is a file
        pdf = pdfplumber.open(f)
        page = pdf.pages[0]
        text = page.extract_text()
        pdf.close()
        try:
            ship_num = regexp.search(text).group(1) #
            ship_num = ".".join(re.findall(r'\d+', ship_num))
            f1 = f.rpartition('/')[0] + '/Transport_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass
        try:
            ship_num = regexp_2.search(text).group(1) #
            f1 = f.rpartition('/')[0] + '/Transport_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass

In [79]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.serialization import AzureJSONEncoder
#from azure.ai.formrecognizer import AnalyzeResult
import json
import glob, os

endpoint = "https://for-transporean.cognitiveservices.azure.com/"
key = "e33ff464f80b4d2d98c84c0bc1fb291f"

path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/"
path_destination = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/"
i = 1

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key))

list_of_files = glob.glob(path_dir+'*.pdf')
for file in list_of_files:
    file = str(file)
    path_to_sample_documents = file
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f
        )
    result = poller.result()

    analyze_result_dict = result.to_dict()
    with open(path_destination + 'data' + "_" +str(i)+ ".json", 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder)
    i += 1


In [80]:
import re #parse json files
import pandas as pd

path_dir = 'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/'
cols = ['Cust Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'PRODUCT', 'LOADING_PLANT_MV3', 'DELIVERY', 'SAP Order No']
df_trans = pd.DataFrame(columns=cols)
#df_trans.columns = cols
lst_trans = []
i = 0

os.chdir(path_dir)
for file in glob.glob("*.json"):
    file = str(file)
    with open(path_dir+file) as file_object:
        data = json.load(file_object) #parse json file to Python obj

    z = data['content']

    trans_num = re.search(r'Transport No. (.*?) Delivery', z).group(1) #extract str between two strings
    #trans_ID = re.search(r'ID (.*?) Weight', z, flags=re.MULTILINE).group(1)
    #weight = re.search(r'Weight (.*?)kg', z).group(1) #
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    print(cust[0])
    try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
    except:
            pass
    
    if cust[0] == "SINGAPORE ADHESION PLANT":
                sap_num = re.findall(r"(\d+)\nnormalorder", z)[0]
                print(sap_num)
    if cust[0] == "CHAMPIONX SG SERVICE PTE LTD":
        sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
        #sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
        print(sap_num)
    try:
        shipper = re.search(r'(.*?) Scheduling', z).group(1) #
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1) ## get numbers after str

    except:
        shipper = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG', z)[0]       
              
    po_num = re.findall(r'(?s)Purchase order No.\n(.*?)\n[^\S\r\n]*Gross Weight', z)
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    origin = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG-', z) #MV2
    plant = re.findall(r'(?s)Division\n(.*?)\n[^\S\r\n]*Product', z)
    #vehicle = re.search(r'AP - (.*?) T', z).group(1)
    #vehicle = str(vehicle) + 'Truck'
    
    #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
    deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
    deli_date = deli_date[0].split(' ', 1)[0]
    if cust[0] == 'EXXONMOBIL CHEMICAL COMPANY':
        deli_date = deli_date.split('\n')[0]
        
    if cust[0] == 'RESPACK MANUFACTURING SDN BHD':
        p = re.compile('MY-([\S\s]*)\n', re.MULTILINE) #Respack
        deli_date = re.findall(p, respack)[0].split('\n')[1].split(' ')[0]
    try:
        shipping_num = re.findall(r'(?s)Order type\n(.*?)\n[^\S\r\n]*Shipping ', z)[0] #
        shipping_num = ".".join(re.findall(r'\d+', shipping_num))
    except:
        pass
    try:
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1)
    except:
        pass
    prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
    
    try:
        deadline = re.search(r'Deadline (.*?) ', z).group(1)
        tifu_patt = r'(?<=General Text)(\s.+)?\nAdditional'
        tifu = re.findall(tifu_patt, z, re.DOTALL)[0]
        tifu = os.linesep.join([s for s in tifu.splitlines() if s])
        first, *others = tifu.splitlines()
    except:
        pass  
    try:
        ord_rem = cust[0] + prod_info[0]
    except:
        pass
    #lst_trans.extend([shipping_num, ord_rem, cust, deli_date, weight])
    if origin[0] =='M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant':
        df_trans.loc[i, 'MV2'] = 'EXX004'
    else:
        df_trans.loc[i, 'MV2'] = 'EXX003'
    if cust[0] == 'THE POLYOLEFIN - PULAU AYER MERBAU':
        try:
            cust_1 = cust[1].split('\n')[-1].split(', SG')[0]
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
        try:
            cust_1 = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #THE POLYOLEFIN - PULAU AYER MERBAU
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
    else: 
        df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    
    df_trans.loc[i, 'Cust Ref'] = shipping_num
    df_trans.loc[i, 'Order Remarks'] = ord_rem
    df_trans.loc[i, 'Transport No'] = trans_num
    df_trans.loc[i, 'Coll Date'] = deli_date
    df_trans.loc[i, 'DEPOT_MV2'] = origin[0]
    try:
        df_trans.loc[i, 'PRODUCT'] = prod_info[0]
    except:
        pass
    df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    df_trans.loc[i, 'DELIVERY'] = deli_date
    df_trans.loc[i, 'SAP Order No'] = sap_num
    df_trans
    i += 1
    
df_trans

MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)
12726480
MITSUI ELASTOMERS SINGAPORE PTE LTD
12720817
MITSUI ELASTOMERS SINGAPORE PTE LTD
12720816
THE POLYOLEFIN - PULAU AYER MERBAU
12726160
MEGACHEM MANUFACTURING PTE LTD
5950321
CHAMPIONX SG SERVICE PTE LTD
5955533


,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,PRODUCT,LOADING_PLANT_MV3,DELIVERY,SAP Order No,MV2
0,1003982065,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)EXXSO...,6108322714,2023-01-30,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-01-30,12726480,EXX003
1,1003982230,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108324473,2023-01-31,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-31,12720817,EXX004
2,1003982292,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108324472,2023-01-31,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-31,12720816,EXX004
3,1003982293,THE POLYOLEFIN - PULAU AYER MERBAUEXXSOL HEXAN...,6108324476,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,2023-01-31,12726160,EXX003
4,1003982307,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108324475,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,2023-01-31,5950321,EXX003
5,1003982760,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328089,2023-02-02,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,NaN,CHAMPIONX SG SERVICE PTE LTD,2023-02-02,5955533,EXX003


In [81]:
temp = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Template_Local.csv") #template for upload
tables = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Tables.csv") #tables for mapping

In [82]:
temp_upload = pd.concat([temp, df_trans], ignore_index=True)

temp_upload = pd.merge(left=temp_upload, right=tables, how='left') #merge df 
temp_upload = temp_upload.fillna(method='ffill')
#temp_upload = temp_upload.drop(['MV3'], axis=1)
temp_upload = temp_upload.rename(columns={'Code': 'MV3'}) #for upload to TMS

In [83]:
colss = ['Customer No', 'Order Remarks', 'Delivery Type No', 'ISO abbreviation', 'Cargo Type No', 'MV1', 'MV2', 'MV4', 'MV5', 'MV6',
         'Plant Code', 'Special Handling No', 'Destination', 'MV3', 'Delivery Type']
temp_upload_1 = temp_upload.drop(columns=colss, axis=1)
temp_upload_1 = temp_upload_1.rename(columns={'LOADING_PLANT_MV3': 'LOADING PLANT', 'DEPOT_MV2': 'DEPOT'})
colss_reord = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'SAP Order No']
temp_upload_1 = temp_upload_1[colss_reord] #to compare with master
temp_upload_1 = temp_upload_1.reset_index().drop(columns='index')
#temp_upload_1

In [84]:
temp_upload_1['Coll Date'] = pd.to_datetime(temp_upload_1['Coll Date']).dt.strftime('%d/%m/%Y') #format datetime
temp_upload_1['DELIVERY'] = pd.to_datetime(temp_upload_1['DELIVERY']).dt.strftime('%d/%m/%Y')

temp_upload_1.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
temp_upload_1

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003982292,6108324472,31/01/2023,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023,12720816
2,1003982230,6108324473,31/01/2023,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023,12720817
3,1003982293,6108324476,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,31/01/2023,12726160
4,1003982065,6108322714,30/01/2023,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),30/01/2023,12726480
5,1003982307,6108324475,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,31/01/2023,5950321
6,1003982760,6108328089,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955533


In [90]:
columns = ['NO', 'SH Reference', 'Job Type', 'Shipper', 'Permit', 'ETA Date', 'QTY', 'Cont Size', 'Cont Type',
          'Cont No', 'SEAL', 'TIME SLOT', 'REMARKS', 'TD- STATUS']
master = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/EM master file.csv")
master = master.drop(columns=columns, axis=1)
master = master.iloc[:, 1:]
master = master.reset_index().drop(columns='index')

from datetime import datetime
try:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
except:
    pass
#master['EM SO# Ref'] = master['EM SO# Ref'].astype(int)

In [91]:
temp_upload_cust = temp_upload_1['SAP Order No'].tolist()
#temp_upload_cust = list(map(int, temp_upload_cust)) #convert list items to int

master = master[master['EM SO# Ref'].isin(temp_upload_cust)] #select rows based on list
#master = master[master['SAP Order No'].isin(temp_upload_cust)]
col_mas = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'EM SO# Ref'] #'EM SO# Ref'
master = master[col_mas]
master.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)

master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master.index = np.arange(1, len(master)+1)

if len(master) == 0:
    print('\033[1m'+ '\033[94m'+ 'All download Transports are new')
master

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003982292,6108324472,31/1/2023,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,31/1/2023,12720816


In [94]:
if len(master) != 0:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [95]:
df_sap = master.copy() #transports with changes
df_sap.rename(columns={'SAP Order No': 'SAP_Order_No'}, inplace=True)
sap = df_sap[df_sap.duplicated('SAP_Order_No', keep=False)].groupby('SAP_Order_No')['Coll Date'].apply(list).reset_index()
lst_sap = sap['SAP_Order_No'].tolist()
sap_1 = df_sap[df_sap["SAP_Order_No"].isin(lst_sap)]

if len(lst_sap) != 0:
    for i in range(len(sap)):
        print(sap.at[i, 'Coll Date'])
        print(lst_sap)
        #print(sap_1)
else:
    print('No transports changes')

No transports changes


C:\Users\john.tan\AppData\Local\Temp\ipykernel_11812\1141463110.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sap = df_sap[df_sap.duplicated('SAP_Order_No', keep=False)].groupby('SAP_Order_No')['Coll Date'].apply(list).reset_index()


In [96]:
from datetime import datetime
for i in range(len(sap)):
    sap_order = sap.at[i, 'SAP_Order_No']
    coll_date = sap.at[i, 'Coll Date']
    coll_date_lst = [datetime.strptime(coll, "%Y-%m-%d").date() for coll in coll_date] #convert elem in list to date
    print(sap_order + ' Latest collection date: '+ max(coll_date_lst).strftime("%Y-%m-%d"))

In [99]:
#new transports works 16/01/2023!
uploaded_trans = df_trans['SAP Order No'].tolist()
master_trans = master['SAP Order No'].tolist()

uploaded_trans = np.unique(uploaded_trans).astype(float)
master_trans = np.unique(master_trans).astype(float)

print('New transport SAP Order Number ')
#np.setdiff1d(uploaded_trans, master_trans).astype(int)
uploaded_trans = [x for x in uploaded_trans if x not in master_trans]
uploaded_trans = list(map(int, uploaded_trans)) #convert to int
uploaded_trans = [ele for ele in uploaded_trans if ele > 0]
print('\033[1m'+ '\033[34m'+ str(uploaded_trans))

new_trans = [] #convert list of int to str
for x in uploaded_trans:
    str_x = "% s" % x
    new_trans.append(str_x)

df_newtrans = df_trans[df_trans['SAP Order No'].isin(new_trans)]
df_newtrans

New transport SAP Order Number 
[12720817, 12726160, 12726480, 5950321, 5955533]


,Cust Ref,Order Remarks,Transport No,Coll Date,DEPOT_MV2,PRODUCT,LOADING_PLANT_MV3,DELIVERY,SAP Order No,MV2
0,1003982065,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)EXXSO...,6108322714,2023-01-30,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),2023-01-30,12726480,EXX003
1,1003982230,MITSUI ELASTOMERS SINGAPORE PTE LTDEXXSOL HEXA...,6108324473,2023-01-31,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2023-01-31,12720817,EXX004
3,1003982293,THE POLYOLEFIN - PULAU AYER MERBAUEXXSOL HEXAN...,6108324476,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,2023-01-31,12726160,EXX003
4,1003982307,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108324475,2023-01-31,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,2023-01-31,5950321,EXX003
5,1003982760,MEGACHEM MANUFACTURING PTE LTDSOLVESSO 150 AP VL,6108328089,2023-02-02,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,NaN,CHAMPIONX SG SERVICE PTE LTD,2023-02-02,5955533,EXX003


In [100]:
#comp (iterate loading plant) with color, write to excel works 15/1/2023!
#import numpy as np

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

def repeatRows(d, n):
    return pd.concat([d]*n)

temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
            
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #print(df_comp)
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y-%m-%d"), mode='w', if_sheet_exists='replace', engine='openpyxl')

In [ ]:
class MyList(list):
    def __contains__(self, typ):
        return any(isinstance(val, typ) for val in self)

x = MyList(uploaded_trans)
print (int in x)

In [688]:
#optional
def repeatRows(d, n):
    return pd.concat([d]*n)

n = abs(len(master) - len(temp_upload_1)) #ensure both df same len
if n!= 0:
    if len(master) > len(temp_upload_1):
        temp_upload_1 = pd.concat([temp_upload_1, repeatRows(temp_upload_1[-1:], n)])
        temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
        #print(len(temp_upload_1))
    else:
        master = pd.concat([master, repeatRows(master[-1:], n)])
        master.index = np.arange(1, len(master)+1)
        #print(len(master))
else:
    print('No existing transports to compare')

In [689]:
df_all = pd.concat([temp_upload_1, master], axis='columns', keys=['First', 'Second'])
df_comp = df_all.swaplevel(axis='columns')[temp_upload_1.columns[1:]]

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

master = master.dropna(subset=['Transport No','Coll Date'], how='all')
temp_upload_1 = temp_upload_1.dropna(subset=['Transport No','Coll Date'], how='all')

df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
#df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
df_comp

C:\Users\john.tan\AppData\Local\Temp\ipykernel_7004\2874637043.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)


In [690]:
master.compare(temp_upload_1, keep_shape=True, keep_equal=True)

Cust Ref             Transport No               Coll Date              \
         self       other         self       other        self       other   
1  1003982231  1003982292   6108324474  6108324472  2023-01-30  31/01/2023   
2  1003982231  1003982230   6108324474  6108324473  2023-01-31  31/01/2023   
3  1003982231  1003982293   6108324474  6108324476  2023-01-31  31/01/2023   
4  1003982231  1003982231   6108324474  6108324474  2023-01-31  31/01/2023   
5  1003982231  1003982307   6108324474  6108324475  2023-01-31  31/01/2023   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
3  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
4  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
5  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                                     PRODUCT  \
                                               other                    self   
1  M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...  EXXSOL HEXANE SP AP VL   
2  M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...  EXXSOL HEXANE SP AP VL   
3  T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...  EXXSOL HEXANE SP AP VL   
4  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  EXXSOL HEXANE SP AP VL   
5  T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...  EXXSOL HEXANE SP AP VL   

                                          LOADING PLANT  \
                    other                          self   
1  EXXSOL HEXANE SP AP VL  BASF SOUTH EAST ASIA PTE LTD   
2  EXXSOL HEXANE SP AP VL  BASF SOUTH EAST ASIA PTE LTD   
3  EXXSOL HEXANE SP AP VL  BASF SOUTH EAST ASIA PTE LTD   
4  EXXSOL HEXANE SP AP VL  BASF SOUTH EAST ASIA PTE LTD   
5      SOLVESSO 150 AP VL  BASF SOUTH EAST ASIA PTE LTD   

                                          DELIVERY             SAP Order No  \
                                 other        self       other         self   
1  MITSUI ELASTOMERS SINGAPORE PTE LTD  2023-01-31  31/01/2023     12734190   
2  MITSUI ELASTOMERS SINGAPORE PTE LTD  2023-01-31  31/01/2023     12734190   
3   THE POLYOLEFIN - PULAU AYER MERBAU  2023-01-31  31/01/2023     12734190   
4         BASF SOUTH EAST ASIA PTE LTD  2023-01-31  31/01/2023     12734190   
5       MEGACHEM MANUFACTURING PTE LTD  2023-01-31  31/01/2023     12734190   

             
      other  
1  12720816  
2  12720817  
3  12726160  
4  12734190  
5   5950321

In [101]:
#df_trans = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/temp_upload.csv") #Textract
trans_1 = df_trans[df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
#trans_1 = df_trans[~df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
vopak = trans_1.copy()
#vopak = vopak.iloc[:, 1:] #drop first col
vopak.reset_index(drop=True, inplace=True)

#vopak.insert(loc=0, column='Transporter Name', value='SH COGENT LOGISTICS PTE LTD')
#vopak.insert(loc=0, column='Customer Name', value='ExxonMobil Chemical Asia Pacific')

vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'MV2'])
#vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'MV2'])
#vopak.rename(columns={'Cust Ref': 'Customer Reference Number', 'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)
vopak.rename(columns={'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)

In [102]:
vopak

,Cust Ref,Schedule Date,Product name,SAP Order No
0,1003982065,2023-01-30,EXXSOL HEPTANE AP VL,12726480
1,1003982293,2023-01-31,EXXSOL HEXANE SP AP VL,12726160
2,1003982307,2023-01-31,SOLVESSO 150 AP VL,5950321
3,1003982760,2023-02-02,NaN,5955533


In [103]:
#Vopak email within 10 working days
from datetime import datetime, timedelta 
#acc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Accepted_transports.csv")
vopak['Schedule Date'] = pd.to_datetime(vopak['Schedule Date'])
curr = datetime.now()
end = curr + timedelta(days=14)
start = curr - timedelta(days=20)

vopak_1 = vopak[vopak['Schedule Date'].between(start, end)]
lst_vopak = vopak_1['Cust Ref'].tolist()
if len(vopak_1) != 0:
    print(vopak_1)
else:
    print('No Vopak transports in next 10 business days')

send_vopak = vopak_1.copy() #make copy of vopak_1 df

     Cust Ref Schedule Date            Product name SAP Order No
0  1003982065    2023-01-30    EXXSOL HEPTANE AP VL     12726480
1  1003982293    2023-01-31  EXXSOL HEXANE SP AP VL     12726160
2  1003982307    2023-01-31      SOLVESSO 150 AP VL      5950321


In [122]:
def copyCertainFiles(source_folder, dest_folder, string_to_match, file_type='.pdf'):
    # Check all files in source_folder
    for filename in os.listdir(source_folder):
        # Move the file if the filename contains the string to match
        if file_type == file_type:
            if string_to_match in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

        # Check if the keyword and the file type both match
        elif isinstance(file_type, str):
            if string_to_match in filename and file_type in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

source_folder = r'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/'
dest_folder = r'C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/'
file_type = '.pdf'

for cust_ref in lst_vopak:
    string_to_match = cust_ref
    copyCertainFiles(source_folder, dest_folder, string_to_match, file_type)

#https://stackoverflow.com/questions/61620485/python-if-filename-in-specified-path-contains-string-then-move-to-folder
#move pdf within 10 working days

In [123]:
#outlook to VOPAK
from importlib import reload #send outlook multiple attachments to multiple recipients 28/11/2022 great!!
import win32com.client as win32
import pythoncom
import sys
import os
import pandas as pd
import pdfplumber
from datetime import datetime

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def sendmail(receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear VOPAK
    <br><br>(Test email) Kindly release the booking in the system for the orders.
    <br>Customer Name: ExxonMobil Chemical Asia Pacific
    <br>Transporter Name: SH Cogent Logistics Pte Ltd<br><br>""" + vopak_1.to_html() +'<br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()
    
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/" #copy Vopak pdf to Vopak folder (outlook)
dst = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/"
for file in glob.glob(path_dir+"*.pdf"):
    #file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori.lower() in ['VOPAK', 'vopak']:
            shutil.copy(file, dst) #if contains 'vopak', copy/paste file
            pdf.close()
    except:
        pass
    
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/Chawan_Vopak_email/Send_Vopak/" # Folder of attachments
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address

for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "VOPAK - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    sendmail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John


In [124]:
def newtrans_mail(receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) New transport orders from Transporeon.
    <br><br>""" + df_newtrans.to_html() +"<br><br>SAP Order Number<br>" + str(uploaded_trans) + '<br><br>Download latest pdf from Transporeon<br>' + df_trans.to_html() +'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Comp_color/" # Folder of comp.xlsx
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "New - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    newtrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John


In [413]:
from PyPDF2 import PdfReader, PdfFileReader #TMS file
import re
import glob, os
import pdfplumber
import numpy as np
import pandas as pd

regexp = re.compile("Destination(.*)$", re.MULTILINE)
regexp_poly = re.compile("PENJURU ROAD(.*)$", re.MULTILINE)

tms_lst = []
tms = pd.DataFrame(columns=['Delivery Type No', 'MV2', 'MV3'])
path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Test_2/"
for file in glob.glob(path_dir + "*.pdf"):

    pdf = pdfplumber.open(file)
    pdf_pg_count = len(pdf.pages)
    page = pdf.pages[0]
    #p = page[i]
    text = page.extract_text()
    #print(text)
    
    cust = re.search(regexp, text).group(1).split('SG-')[0] #MV3
    cust = cust.strip()
    conditions=[cust=='EXXONMOBIL', cust=='MITSUI', cust=='SUMITOMO', cust=='A.I.M. CHEMICAL', cust=='CLP INTERNATIONAL', cust=='CHAMPIONX SG', cust=='SINGAPORE', cust=='BASF SOUTH EAST', cust=='ASAHI KASEI', cust=='MEGACHEM', cust=='MSD', cust=='SINGAPORE BUTYL', cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17']
    values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA']
    values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]
    
    MV2 = np.select(conditions, values)
    deli_type = np.select(conditions, values_1)
    print('customer is: '+ str(cust) + ' '+ str(MV2) + ' '+str(deli_type))
    print('------------------')
    if str(cust).strip() == 'THE POLYOLEFIN -':
            
            dest = re.search(regexp_poly, text).group(0)
            if str(dest) =='PENJURU ROAD 59 STREET 2 GATE NO 5 401' or str(dest) == 'PENJURU ROAD 59 STREET 2 GATE NO 23 401':
                MV2 = 'TPCPAM2'
                print('customer is '+ str(dest) + ' '+ str(MV2))
            else:
                MV2 = 'TPCPAM1'
                deli_type = 4
                print('customer is '+ str(dest) + str(MV2) + ' '+ str(deli_type))
            pdf.close()
    tms_lst.extend([[str(deli_type), str(MV2), cust]])
    pdf.close()
#tms_lst

tms = pd.DataFrame(tms_lst, columns=['Delivery_Type', 'MV2', 'Customer_MV3']) #list of lists to df

customer is: MITSUI MITSUI 4
------------------
customer is: MITSUI MITSUI 4
------------------
customer is: EXXONMOBIL EXX006 1
------------------
customer is: ASAHI KASEI AKSS 4
------------------
customer is: MEGACHEM MEGA 1
------------------
customer is: MEGACHEM MEGA 1
------------------
customer is: CHAMPIONX SG CX 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: THE POLYOLEFIN - 0 0
------------------
customer is PENJURU ROAD 59 STREET 2 GATE NO 23 401 TPCPAM2
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: SINGAPORE SAP 1
------------------
customer is: ASAHI KASEI AKSS 4
------------------
customer is: TOTALENERGIES TOTAL 1
------------------
customer is: SUMITOMO SEIKA-17 SEIKA 1
--------------

In [403]:
tms

,Delivery_Type,MV2,Customer_MV3
0,4,MITSUI,MITSUI
1,4,MITSUI,MITSUI
2,1,EXX006,EXXONMOBIL
3,4,AKSS,ASAHI KASEI
4,1,MEGA,MEGACHEM
5,1,MEGA,MEGACHEM
6,1,CX,CHAMPIONX SG
7,1,SAP,SINGAPORE
8,1,SAP,SINGAPORE
9,1,SAP,SINGAPORE


In [107]:
import xlsxwriter

writer = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Style.xlsx", engine='xlsxwriter')
df_comp.to_excel(writer, sheet_name='Comp') #write to excel with color
worksheet1 = writer.sheets['Comp']
worksheet1.set_tab_color('blue')
writer.save()
writer.close()

C:\Users\john.tan\AppData\Local\Temp\ipykernel_11812\3457223958.py:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\users\john.tan\appdata\local\programs\python\python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [117]:
def dataframe_difference(df1, df2, which):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    #diff_df.to_csv('data/diff.csv')
    return diff_df

new_upload = dataframe_difference(temp_upload_1, master, 'left_only')
new_upload = new_upload.rename(columns={'_merge': 'New_Download'})
new_upload
#temp_upload_2 = temp_upload_1.groupby('LOADING PLANT').apply(lambda x: x.sort_values('SAP Order No'))

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No,New_Download
0,1003982292,6108324472,31/01/2023,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023,12720816,left_only
1,1003982230,6108324473,31/01/2023,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023,12720817,left_only
2,1003982293,6108324476,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,31/01/2023,12726160,left_only
3,1003982065,6108322714,30/01/2023,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),30/01/2023,12726480,left_only
4,1003982307,6108324475,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,31/01/2023,5950321,left_only
5,1003982760,6108328089,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955533,left_only


In [465]:
tp = temp_upload_1.groupby('LOADING PLANT')    
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df

ms = master.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

def repeatRows(d, n):
    return pd.concat([d]*n)

for i in range(min(len(lst_tp), len(lst_ms))):
        
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:
        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
            #print(len(lst_tp[i]))
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            #print(len(lst_ms[i]))
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        print(lst_ms[i].compare(lst_tp[i], keep_shape=True, keep_equal=True)) #compare side by side
        
#https://stackoverflow.com/questions/23691133/split-pandas-dataframe-based-on-groupby

     Cust Ref             Transport No              Coll Date              \
         self       other         self       other       self       other   
1  1003974560  1003974560   6108272471  6108272471  13/1/2023         NaN   
2  1003978428  1003974560   6108298482  6108272471  18/1/2023  01/11/2023   
3  1003976739  1003978428   6108288398  6108298482  19/1/2023  13/01/2023   
4  1003976739  1003979353   6108288398  6108305780  19/1/2023         NaN   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
3  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
4  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                                     PRODUCT  \
                                               other                    self   
1  T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...  EXXSOL HEXANE SP AP VL   
2 

In [474]:
#alternative comparison
tp6 = temp_upload_1.copy()
ms6 = master.copy()
tp6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)
ms6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)

diffs = set(ms6.iloc[:, :3].apply(tuple, 1)) \
- set(tp6.apply(tuple, 1))
lst_diffs = list(diffs)

50


In [486]:
lst_tp[11].index = np.arange(1, len(lst_tp[11])+1)
lst_ms[11].index = np.arange(1, len(lst_ms[11])+1)
lst_ms[11].compare(lst_tp[11], keep_shape=True, keep_equal=True)

Cust Ref             Transport No              Coll Date              \
         self       other         self       other       self       other   
1  1003977817  1003976424   6108295225  6108286474  11/1/2023  16/01/2023   
2  1003977817  1003977817   6108295225  6108295225  11/1/2023  18/01/2023   

                                        DEPOT  \
                                         self   
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59   

                                                           PRODUCT  \
                                               other          self   
1  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  SPECTRASYN 4   
2  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  SPECTRASYN 4   

                                                   LOADING PLANT  \
        other                                               self   
1  MCP2542 VL  TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...   
2  MCP2542 VL  TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EA...   

                                                       DELIVERY              \
                                               other       self       other   
1  TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE E...  11/1/2023  10/01/2023   
2  TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE E...  11/1/2023  11/01/2023   

  SAP Order No            
          self     other  
1     12728630  12727043  
2     12728630  12728630

In [42]:
#import numpy as np #comp in color to excel works 17/1/2023!
temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])           
          
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        
            
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #df_comp
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')
                

    if n ==0:    
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
                
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')

     Cust Ref Transport No   Coll Date  \
1  1003982231   6108324474  31/01/2023   
2  1003982231   6108324474  31/01/2023   

                                               DEPOT                 PRODUCT  \
1  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  EXXSOL HEXANE SP AP VL   
2  T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...  EXXSOL HEXANE SP AP VL   

                  LOADING PLANT    DELIVERY SAP Order No  
1  BASF SOUTH EAST ASIA PTE LTD  31/01/2023     12734190  
2  BASF SOUTH EAST ASIA PTE LTD  31/01/2023     12734190  
     Cust Ref Transport No   Coll Date  \
1  1003982231   6108324474  2023-01-30   
2  1003982231   6108324474  2023-01-31   

                                        DEPOT                 PRODUCT  \
1  Vopak (Penjuru) Terminal \nPENJURU ROAD 59  EXXSOL HEXANE SP AP VL   
2  Vopak (Penjuru) Terminal \nPENJURU ROAD 59  EXXSOL HEXANE SP AP VL   

                  LOADING PLANT    DELIVERY SAP Order No  
1  BASF SOUTH EAST ASIA PTE LTD  2023-01-31     1

C:\Users\john.tan\AppData\Local\Temp\ipykernel_11812\2786167780.py:37: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)


In [127]:
#compare Upcoming with master 18/01/2023
upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Upcoming.csv", usecols=['Order No'])
upc.rename(columns={'Order No': 'SAP Order No'}, inplace=True) #'Deliv Date': 'DELIVERY'
#upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/EM master file.csv", usecols=['EM SO# Ref'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master_upc.index = np.arange(1, len(master_upc)+1)

sap_master_upc = master_upc['SAP Order No'].tolist()

master_upc_1 = master_upc.copy()
master_upc_1['SAP Order No'] = master_upc_1['SAP Order No'].astype(np.int64) #col to int
upOnly = dataframe_difference(upc, master_upc_1, 'left_only')

upOnly.rename(columns={"_merge": "New_Transport"}, inplace=True)
upOnly['New_Transport'] = upOnly['New_Transport'].replace('left_only', 'new upcoming')
upOnly.index = np.arange(1, len(upOnly)+1)
print('\033[94m' + 'Transports from Upcoming not in MasterFile')
upOnly

Transports from Upcoming not in MasterFile


,SAP Order No,New_Transport
1,5944859,new upcoming
2,12720815,new upcoming
3,12726060,new upcoming
4,12732175,new upcoming
5,12732176,new upcoming
6,12732177,new upcoming
7,12732178,new upcoming
8,4452380265,new upcoming
9,4452380265,new upcoming
10,4452380265,new upcoming


In [108]:
#compare Upcoming with master (development)
upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Upcoming.csv", usecols=['Order No', 'Deliv Date', 'Customer Name'])
upc.rename(columns={'Order No': 'SAP Order No', 'Customer Name': 'LOADING', 'Deliv Date': 'DELIVERY'}, inplace=True)
upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/EM master file.csv", usecols=['EM SO# Ref', 'DELIVERY', 'LOADING PLANT'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No', 'LOADING PLANT': 'LOADING'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')

sap_master_upc = master_upc['SAP Order No'].tolist()
#print(sap_master_upc) #list of str
#print(lst_upc) #list of int

upc['SAP Order No'] = upc['SAP Order No'].astype(str) #convert col to str
upc_notin_master = upc[~upc['SAP Order No'].isin(sap_master_upc)] #select Upcoming not in Master
upc_notin_master.index = np.arange(1, len(upc_notin_master)+1)
#print('Upcoming transports not in Master file')
#print(upc_notin_master)

idx1 = set(upc.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
idx2 = set(upc_notin_master.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
upc_in_master = pd.DataFrame(list(idx1 - idx2), columns=upc.columns) #select Upcoming in Master

upc_in_master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
upc_in_master.index = np.arange(1, len(upc_in_master)+1)
print('Upcoming transports in Master file')
print(upc_in_master)

master_upc.index = np.arange(1, len(master_upc)+1)
print('Master file')
print(master_upc)

#diffs = set(master_upc.iloc[:, :3].apply(tuple, 1)) \
                       #-  set(upc.apply(tuple, 1))
#print('Difference in master ')
#print(diffs)

diffs_2 = set(upc.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master_upc.apply(tuple, 1))
#print('New entry in Upcoming transports')
DF_diffs_2 = pd.DataFrame(data=list(diffs_2), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
#DF_diffs_2
#https://stackoverflow.com/questions/46635439/compare-rows-of-two-data-frames-with-unequal-lengths
#https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe

Upcoming transports in Master file
   SAP Order No                              LOADING    DELIVERY
1      12710740    THE POLYOLEFIN -PULAU AYER MERBAU  17/01/2023
2      12720808  MITSUI ELASTOMERS SINGAPORE PTE LTD  13/01/2023
3      12720809  MITSUI ELASTOMERS SINGAPORE PTE LTD  13/01/2023
4      12720810  MITSUI ELASTOMERS SINGAPORE PTE LTD  17/01/2023
5      12720811  MITSUI ELASTOMERS SINGAPORE PTE LTD  17/01/2023
6      12720812  MITSUI ELASTOMERS SINGAPORE PTE LTD  20/01/2023
7      12720813  MITSUI ELASTOMERS SINGAPORE PTE LTD  20/01/2023
8      12720814  MITSUI ELASTOMERS SINGAPORE PTE LTD  27/01/2023
9      12726058         ASAHI KASEI SYNTHETIC RUBBER  13/01/2023
10     12726059         ASAHI KASEI SYNTHETIC RUBBER  18/01/2023
11     12726152   THE POLYOLEFIN - PULAU AYER MERBAU  27/01/2023
12     12726478               MSD INTERNATIONAL GMBH  16/01/2023
13     12726479               MSD INTERNATIONAL GMBH  19/01/2023
14     12727051   THE POLYOLEFIN - PULAU AYER MERBAU  2

In [129]:
from importlib import reload #send outlook multiple recipients 18/1/2023 great!!
import win32com.client as win32
import pythoncom
import sys
import os
import pandas as pd
import pdfplumber
from datetime import datetime

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def upctrans_mail(receiver, attachment, subject):
    receiver = receiver
    #attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) EM MasterFile.
    <br><br>""" + master_upc.to_html() +"<br><br>Upcoming Transports not in MasterFile<br>" + upOnly.to_html()+'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    #for i in range(len(attachment)):
        #mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Upcoming/" # Folder of comp.xlsx
addr = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "Upcoming - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    upctrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

John


In [110]:
temp_upload_diff = temp_upload_1[['SAP Order No', 'LOADING PLANT', 'DELIVERY']]
temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)
diffs_3 = set(temp_upload_diff.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master.apply(tuple, 1))
print('New entry in Uploaded transports')
DF_diffs_3 = pd.DataFrame(data=list(diffs_3), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
DF_diffs_3 #Upcoming not in master

New entry in Uploaded transports


C:\Users\john.tan\AppData\Local\Temp\ipykernel_11812\2230247851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)


,SAP Order No,LOADING,DELIVERY
0,12726160,THE POLYOLEFIN - PULAU AYER MERBAU,31/01/2023
1,12726480,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),30/01/2023
2,5950321,MEGACHEM MANUFACTURING PTE LTD,31/01/2023
3,12720817,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023
4,5955533,CHAMPIONX SG SERVICE PTE LTD,02/02/2023
5,12720816,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023


In [599]:
#df3 = df2.reindex_like(df1)
#df1[(df3['a'].isin(df1['a'])) & (df1['b'] <= df3['b'])]
comp_6 = temp_upload_1.reindex_like(master)
master[(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['PRODUCT'] <= comp_6['PRODUCT'])]
#master[np.logical_not(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['DEPOT'] <= comp_6['DEPOT'])]

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003980836,6108314616,2023-01-27,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,2023-01-27,12720814


In [111]:
master

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003982292,6108324472,2023-01-31,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,2023-01-31,12720816


In [112]:
temp_upload_1

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1003982292,6108324472,31/01/2023,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023,12720816
2,1003982230,6108324473,31/01/2023,M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Flu...,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,31/01/2023,12720817
3,1003982293,6108324476,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,EXXSOL HEXANE SP AP VL,THE POLYOLEFIN - PULAU AYER MERBAU,31/01/2023,12726160
4,1003982065,6108322714,30/01/2023,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pen...,EXXSOL HEPTANE AP VL,MSD INTERNATIONAL GMBH (SINGAPORE BRANCH),30/01/2023,12726480
5,1003982307,6108324475,31/01/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,MEGACHEM MANUFACTURING PTE LTD,31/01/2023,5950321
6,1003982760,6108328089,02/02/2023,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Ter...,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,02/02/2023,5955533


In [113]:
import xlsxwriter #write to excel new transports 50

#writer1 = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/temp_upload.xlsx", engine='xlsxwriter')
#temp_upload_1.to_excel(writer1, sheet_name='temp_upload') #write to excel with color
#worksheet1 = writer1.sheets['temp_upload']
#worksheet1.set_tab_color('blue')
#writer1.save()
#writer1.close()

#writer2 = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/master.xlsx", engine='xlsxwriter')
#master.to_excel(writer2, sheet_name='master')
#worksheet2 = writer2.sheets['master']
#worksheet2.set_tab_color('purple')
#writer2.save()
#writer2.close()

with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Test/Comp_color/NewTransports_50.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer3:
    sheetName = 'NewTransports_' + datetime.now().strftime("%Y_%m_%d %H%M")
    df_trans.to_excel(writer3, sheet_name=sheetName, engine='openpyxl')
    worksheet3 = writer3.sheets[sheetName]
    writer3.save()

print('Process completed')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_11812\93389021.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer3.save()
